In [ ]:
import pandas as pd
import surprise as sp
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import KFold
from surprise import accuracy
from surprise.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
df = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/suprise_df.csv")
df5 = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/suprise_5.csv")
dfb = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/suprise_b.csv")

In [ ]:
df

In [ ]:
# Change it back to countries
#melted.iloc[:, 1] = labelencoder.inverse_transform(melted.iloc[:, 1])
labelencoder = LabelEncoder()
df.iloc[:, 1] = labelencoder.fit_transform(df.iloc[:, 1])
df

In [ ]:
df.city.value_counts()

In [ ]:
reader= Reader(rating_scale=(1.0, 3.0))
data = sp.Dataset.load_from_df(df[['user_id', 'city', 'rank']], reader)

reader5= Reader(rating_scale=(1.0, 5.0))
data5 = sp.Dataset.load_from_df(df5[['user_id', 'city', 'rank']], reader5)

readerb = Reader(rating_scale=(1.0, 5.0))
datab = sp.Dataset.load_from_df(dfb[['user_id', 'city', 'rank']], readerb)

In [ ]:
trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
trainset

In [ ]:
benchmark = []
# Iterate over all algorithms
for algorithm in [sp.SVD(), sp.NMF(), sp.NormalPredictor(), sp.KNNBasic(), sp.KNNWithZScore(), sp.CoClustering(),
                 sp.SlopeOne(), sp.KNNBaseline(), sp.KNNWithMeans(), sp.SVDpp(), sp.BaselineOnly()]:
# Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
# Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]],index=['Algorithm']))
    benchmark.append(tmp)

In [ ]:
benchmark = pd.DataFrame(benchmark)
benchmark

### GridSearch CV 

With SVD

In [ ]:
param_grid = {'n_factors': [25, 30, 35, 40, 100], 'n_epochs': [15, 20, 25], 'lr_all': [0.001, 0.003, 0.005, 0.008],               'reg_all': [0.08, 0.1, 0.15, 0.02]}
gs = GridSearchCV(sp.SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data) 
algo = gs.best_estimator['rmse']
print(gs.best_score['rmse']) 
print(gs.best_params['rmse'])

In [ ]:
algo2 = sp.SVD(n_factors= 25, n_epochs = 15, lr_all = .001, reg_all = .08)
kf = KFold(n_splits=3)


for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo2.fit(trainset)
    predictions = algo2.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
predictions

In [ ]:
sim = {'name': 'pearson_baseline', 'user_based': True, 'min_support': 5}
algo = sp.KNNBasic(k_min = 2, sim_options = sim)
kf = KFold(n_splits=3)

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
predictions